In [1]:
import simpy
import tkinter as tk
from tkinter import messagebox

# Sample venue booking data
venues = {
    'Auditorium': {
        'bookings': {
            '2025-04-20': ['09:00-11:00'],
            '2025-04-21': ['14:00-16:00']
        }
    },
    'LT-1': {
        'bookings': {
            '2025-04-20': ['11:00-13:00'],
            '2025-04-22': ['16:00-18:00']
        }
    },
    'CR-1': {
        'bookings': {
            '2025-04-21': ['09:00-11:00'],
            '2025-04-22': ['14:00-16:00']
        }
    }
}
time_slots = ['09:00-11:00', '11:00-13:00', '14:00-16:00', '16:00-18:00']

# SimPy event-driven simulation logic
def check_availability_simulation(env, venue, date, time_slot):
    # Check availability in the SimPy simulation environment
    yield env.timeout(1)  # Simulate the time taken to check availability

    # Venue logic: if slot is already booked, it cannot be booked
    if venue in venues:
        booked_slots = venues[venue]['bookings'].get(date, [])
        if time_slot in booked_slots:
            return False  # Slot is booked
        else:
            return True  # Slot is available
    else:
        return False  # Invalid venue

def book_venue_simulation(env, venue, date, time_slot):
    # Simulate the booking process
    availability = yield env.process(check_availability_simulation(env, venue, date, time_slot))

    if availability:
        if venue in venues:
            if date not in venues[venue]['bookings']:
                venues[venue]['bookings'][date] = []
            venues[venue]['bookings'][date].append(time_slot)
            return True  # Booking success
    return False  # Booking failure

# Function to handle the booking request using SimPy
def handle_booking_request(venue, date, time_slot):
    env = simpy.Environment()  # Create a SimPy environment for the simulation
    booking_result = env.process(book_venue_simulation(env, venue, date, time_slot))

    # Run the simulation
    env.run()

    # Return booking result (True or False)
    return booking_result

# Function to check availability in the GUI
def check_availability():
    venue = venue_var.get()
    date = date_var.get()

    available_slots = []
    for slot in time_slots:
        # SimPy simulation to check availability for each time slot
        available = handle_booking_request(venue, date, slot)
        if available:
            available_slots.append(slot)

    if available_slots:
        available_slots_var.set("\n".join(available_slots))
    else:
        available_slots_var.set("No available slots for this date.")

# Function to book the venue in the GUI
def book_venue():
    venue = venue_var.get()
    date = date_var.get()
    time_slot = time_slot_var.get()

    # SimPy simulation to attempt booking the venue
    booking_result = handle_booking_request(venue, date, time_slot)

    if booking_result:
        messagebox.showinfo("Booking Successful", f"{venue} booked successfully for {date} at {time_slot}.")
    else:
        messagebox.showinfo("Booking Error", "This time slot is already booked!")

# Creating the Tkinter window
root = tk.Tk()
root.title("Venue Booking System")

# Venue selection
venue_var = tk.StringVar()
venue_label = tk.Label(root, text="Select Venue:")
venue_label.grid(row=0, column=0)
venue_options = ['Auditorium', 'LT-1', 'CR-1']
venue_menu = tk.OptionMenu(root, venue_var, *venue_options)
venue_menu.grid(row=0, column=1)

# Date selection
date_var = tk.StringVar()
date_label = tk.Label(root, text="Select Date (YYYY-MM-DD):")
date_label.grid(row=1, column=0)
date_entry = tk.Entry(root, textvariable=date_var)
date_entry.grid(row=1, column=1)

# Button to check availability
check_button = tk.Button(root, text="Check Availability", command=check_availability)
check_button.grid(row=2, column=0, columnspan=2)

# Available slots display
available_slots_var = tk.StringVar()
available_slots_label = tk.Label(root, text="Available Time Slots:")
available_slots_label.grid(row=3, column=0)
available_slots_display = tk.Label(root, textvariable=available_slots_var, width=30, height=4, relief="solid")
available_slots_display.grid(row=3, column=1)

# Time slot selection
time_slot_var = tk.StringVar()
time_slot_label = tk.Label(root, text="Select Time Slot:")
time_slot_label.grid(row=4, column=0)
time_slot_menu = tk.OptionMenu(root, time_slot_var, *time_slots)
time_slot_menu.grid(row=4, column=1)

# Button to book the venue
book_button = tk.Button(root, text="Book Venue", command=book_venue)
book_button.grid(row=5, column=0, columnspan=2)

# Start the Tkinter event loop
root.mainloop()
